In [4]:
import numpy as np

In [5]:
pima = np.genfromtxt('pima-indians-diabetes.data', delimiter=',')

In [29]:
def sghmc(Y, X, stogradU, M, eps, m, theta, C, V):
    n = X.shape[0]
    p = X.shape[1]
    
    # Randomly sample momentum
    r = np.random.multivariate_normal(np.zeros(M.shape[0]),M)[:,np.newaxis]
    
    # Precompute
    B = 0.5 * V * eps
    D = 2*(C-B)*eps
    Minv = np.linalg.inv(M)
    
    # Hamiltonian dynamics
    for i in range(m):
        theta = theta + (eps*np.linalg.inv(M) @ r).ravel()
        r = r - eps*stogradU(theta, Y, X, nbatch) - eps*C @ Minv @ r \
            + np.random.multivariate_normal(np.zeros(M.shape[0]),D)[:,np.newaxis]

    return(theta)

In [49]:
def logistic(x):
    return 1/(1+np.exp(-x))

def stogradU(theta, Y, X, nbatch):
    '''A function that returns the stochastic gradient. Adapted from Eq. 5.
    Inputs are:
        theta, the parameters
        Y, the response
        X, the covariates
        nbatch, the number of samples to take from the full data
    '''
    alpha=5
    n = X.shape[0]
    batch_id = np.random.choice(np.arange(n),nbatch,replace=False)
    
    Y_pred = logistic(X[batch_id,:] @ theta[:,np.newaxis])
    epsilon = (Y[batch_id][:,np.newaxis] - Y_pred)
    
    grad = -n/nbatch * X[batch_id,:].T @ epsilon - theta[:,np.newaxis]/alpha
    
    return grad

In [58]:
X = np.concatenate((np.ones((pima.shape[0],1)),pima[:,0:8]), axis=1)
Y = pima[:,8]

Xs = (X - np.mean(X, axis=0))/np.concatenate((np.ones(1),np.std(X[:,1:], axis=0)))
n, p = X.shape

nsample = 10
nbatch = 768
M = np.identity(p)
C = 0 * np.identity(p)
eps = 0.1
m = 50
V = 0 * np.identity(p)
theta = np.zeros(p)

In [59]:
np.random.seed(2)

# w noise, no mh
samples = np.zeros((nsample, p))

for i in range(nsample):
    theta = sghmc(Y, Xs, stogradU, M, eps, m, theta, C, V)
    samples[i] = theta

In [60]:
theta

array([  1.81964807e+06,   3.35098426e-01,   1.16431918e+00,
        -3.40326103e-01,   2.16111978e-01,   7.05939592e-02,
         5.57263529e-01,   3.30554899e-01,  -7.20119200e-02])

In [23]:
np.std(Xs,axis=0)

array([ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [24]:
np.mean(Xs,axis=0)

array([  0.00000000e+00,  -7.74843153e-17,   3.61400724e-18,
        -1.32724416e-17,   7.76288755e-17,  -5.49329101e-17,
         2.97273780e-15,   1.92438658e-15,   2.19297959e-16])